In [16]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool() # agent가 유튜브 채널 검색 가능
# youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle="@channel_name") #직접 검색할 채널을 넣음

# 1. create agent

# 연구원 - 영상 콘텐츠 생성을 위해 영상 콘텐츠 주제에 대한 정보 수집
researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

# 마케터 - 바이럴하고 유익한 콘텐츠 생성을 위해 아이디어를 내야함
marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

# 작가 - 마케터의 아이디어를 바탕으로 유튜브 영상 스크립트 작성
writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

In [17]:
# 2. Create tasks

# marketer agent에게 특정 산업군에 대한 5가지의 아이디어를 브레인스토밍 하도록 함
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True, # 사람이 개입할 수 있도록 설정 - agent가 다음 task로 넘어갈 때  승인이나 거절 요청 확인 메세지 보임
)
# brainstorm_task에서 괜찮은 1개를 선택하여 작가 agent에게 스크립트 작성을 위한 아이디어를 제공
selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True, # 사람이 개입할 수 있도록 함
    # brain task의 output이 context로 전달됨 - 즉 brain task가 끝나길 기다림
    context=[
        brainstorm_task, 
    ],
    output_file="selection_task.md",
)

#  연구원 agent에게 선택된 1개의 아이디어를 바탕으로 유투브 검색
research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True, # Task들이 동시에 실행 되도록 설정
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

# researcher agent에게 경쟁사의 youtube 채널이나 글을 살펴보도록 함 - 동향 파악 후 그들과 다르게 영상 제작 하고 싶음
competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

# researcher agent에게 다른 업계에서는 핫한 콘텐츠가 있는지 살펴보도록 함
inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

# 작가 agent에게 스크립트 작성 - 경쟁자 영상 참조 및 다른 업계의 영상을 참조 하여 독창적인 스크립트 작성
script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

# 3. Create crew
crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

# 4. Kickoff crew
result = crew.kickoff(
    inputs=dict(
        industry="Hot Sauce",
    ),
)


2024-10-06 12:58:37,777 - 8037344512 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry


> Entering new CrewAgentExecutor chain...
To come up with compelling video ideas for a YouTube channel in the hot sauce industry, it's crucial to understand the current trends, interests, and challenges in the industry. I'll start by asking the Senior Researcher for insights into popular trends and topics in the hot sauce industry that appeal to a young audience. This will help ensure the video ideas are relevant and engaging.

Action: Ask question to co-worker
Action Input: {"question": "What are the current popular trends and topics in the hot sauce industry, especially those that appeal to a young audience?", "context": "I need this information to generate viral video ideas for a YouTube channel focused on the hot sauce industry. The video content should be engaging and relevant to a young audience." "coworker": "Senior Researcher"}
 

I

Exception in thread Thread-50 (_execute):
Traceback (most recent call last):
  File "/Users/sako/.pyenv/versions/3.11.6/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/sako/project/crewAI/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/sako/.pyenv/versions/3.11.6/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sako/project/crewAI/.venv/lib/python3.11/site-packages/crewai/task.py", line 192, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/sako/project/crewAI/.venv/lib/python3.11/site-packages/crewai/agent.py", line 236, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sako/project/crewAI/.venv/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/Users/sa

Thought: I have gathered detailed examples of the "swicy" trend in the food and beverage industry, as well as some insights into its influence on the fashion industry. Now, I will compile these examples into a comprehensive final answer, as required.

Final Answer: Here are examples of articles and videos that relate to the "swicy" concept across different industries:

1. **Food and Beverage Industry**:
   - **New York Post**: 'Swicy' treats are heating up as the summer's big food trend. This article discusses the rise of "swicy" (sweet and spicy) treats in the food and beverage industry, highlighting products like Starbucks' Spicy Lemonade Refreshers and Coca-Cola's spiced variation of its original recipe. The trend is driven by younger generations and a desire to explore unfamiliar ingredients and cuisines. [Link to Article](https://nypost.com/2024/06/03/lifestyle/swicy-treats-are-heating-up-as-the-summers-big-food-trend/)

   - **Nutritional Outlook**: 2024 Flavor trends for food an

'**Title:** Swicy Taste Test: Sweet and Spicy Flavor Explosion\n\n**Thumbnail Prompt:** Create a vibrant, eye-catching thumbnail featuring a mix of sweet and spicy elements like peppers and honey, with bold text saying "Swicy Taste Test!" and images of the featured influencers with surprised expressions. Use bright colors like red, yellow, and orange to evoke the swicy theme.\n\n---\n\n**[INTRO]**\n\n[Scene: Energetic music plays. Quick cuts of colorful swicy dishes and ingredients. The host, smiling, stands in a lively kitchen setting.]\n\nHost: "Welcome back, spice enthusiasts! Today, we\'re diving into the exciting world of \'swicy\' – that\'s sweet and spicy – with a flavor explosion you won\'t want to miss! We\'ve got some amazing guests joining us, so get ready for a swicy adventure like no other!"\n\n**[SECTION 1: Meet the Influencers]**\n\n[Scene: Quick intro clips of each influencer, highlighting their unique personalities and previous food content.]\n\nHost: "Joining us today